# Introduction à JDBC
JDBC est une API intégrée à Java qui permet un accès simple aux Systèmes de Gestion de Dases de Données Relationnelles (SGBDR). Ses objectifs sont d'offrir une vision uniforme des SGDBR notament en s'appuyant sur SQL et en définissant des types adaptés à Java.

C'est une API qui permet de se connecter à une base de données, d'évaluer des requêtes et de parcours l'ensemble des résultats.

Il est aussi adaptable pour utiliser les spécifités d'un SGBDR donné mais au prix de la portabilité.

## The dataset
Pour illuster JDBC ce cours s'appuie sur PostgeSQL et en particulier sur une base de données exemple d'un système de location de DVD (https://www.postgresqltutorial.com/postgresql-sample-database/).
![DVD Rental ER Model](https://sp.postgresqltutorial.com/wp-content/uploads/2018/03/dvd-rental-sample-database-diagram.png).

Définissons tout d'abord simplement une classe Java pour représenter l'une des entité de l'exemple.

In [1]:
%mavenRepo projectlombok.org http://projectlombok.org/mavenrepo
%maven org.projectlombok:lombok:0.9.2

public class Film {
    private int id;
    private String title;
    private String description;
    public Film(int id, String title, String description) {
        this.id = id;
        this.title = title;
        this.description = description;
    }
    public String toString() {return "{id:"+id+",title:'"+title+"',description:'"+description+"'}";}
}

## The JDBC Driver for postgresql
Pour se connecter à un SGBDR spécifique il faut disposer d'une implantation particulière d'un Driver (qui devra être ajouté à l'application). 
Il existe quatre familles de drivers JDBC :
* Type 1 - s'appuie sur un autre driver (par exemple JDBC-ODBC)
* Type 2 - utilise une implatation native d'un driver coté client.
* Type 3 - utilise un middleware pour convertir les appels JDBC en appel s propriétaires du SGBDR.
* Type 4 - offre une connection directe depuis une implantation Java (thin driver).

Le type 4 est le plus courant. Il est indépendant de la plateforme et offre de meilleures performances en se connectant directement à la base de données. 

## Ouverture d'une connexion
Pour ouvrir une connection, il faut charger la classe de l'implantation du Driver et ouvrir une connexion. En théorie, on devrait utiliser ```Class.forName("nom de la classe d'implantation")``` en pratique JDBC le fait automatique en fonction de l'URL lors de la connexion.

La première chose à faire est d'ajouter l'implantation du Driver pour la  base de données visée. Par exemple, en ajoutant à Maven une dépendance comme ci-dessous pour PostgreSQL.

In [2]:
%%loadFromPOM
 <dependency>
            <groupId>org.postgresql</groupId>
            <artifactId>postgresql</artifactId>
            <version>42.2.11</version>
</dependency>

Pour ouvrir une connexion, on utilise une URL dont la forme est spécifique à chaque SGBD par exemple pour [MySQL](https://dev.mysql.com/doc/connector-j/5.1/en/connector-j-usagenotes-connect-drivermanager.html), [PostgreSQL](https://jdbc.postgresql.org/) ou [H2](http://www.h2database.com/html/features.html) (une base de données relationnel en pur Java):
* ```jdbc:mysql://localhost:3306/myDatabase?user=username&password=password```
* ```jdbc:postgresql://localhost/myDatabase```
* ```jdbc:h2:mem:myDatabase```

Les informations d'authentification peuvent parfois être donnés dans l'URL ou lors de l'appel de la méthode ```Connection DriverManager.getConnection(jdbcURL)```.

In [3]:
import java.sql.Connection;
import java.sql.DriverManager;
String jdbcURL="jdbc:postgresql://db:5432/dvdrental?user=postgres&password=changeme";
Connection connection = DriverManager.getConnection(jdbcURL)


## Exécution de requêtes SQL
Pour exécuter des requetes SQL on utilise des instances de la classes ```Statement```.

Il existe trois types de statements :
* statement: requêtes simples
* prepared statement: requêtes précompilées
* callable statement: procédures stockées

Un statement est créé à partir d'une connection.

## Les requêtes simples
La méthode à appeler est différente suivant la nature de la requêtes SQL que l’on veut exécuter :
* Consultation (select)
  * executeQuery() On parcours les t-uples avec un ResultSet

* Mise à jour (update, insert, delete) ou gestion de la base de
données (create table,...)
  * executeUpdate() renvoie le nombre de lignes modifiées

* Type inconnu (ex. donné par un fonction sous forme de String)
ou si la requêtes peut renvoyer plusieurs résultats (procédures
stockées)
  * execute()

In [4]:
import java.sql.Statement;
import java.sql.ResultSet;
// Une requête simple
Statement statement = connection.createStatement();
String query1 = "SELECT * FROM film LIMIT 5";
ResultSet resultSet = statement.executeQuery(query1);

## Parcours des résultats
* executeQuery() retourne de ResultSet
* L’interface ResultSet définit les méthodes pour accéder au valeur des attributs
  * getXXX(int numéroDeColonne)
  * getXXX(String nomDeColonne)
  * XXX désigne le type Java de la valeur que l'on va récupérer (Byte, Boolean, AsciiStream, Short, String UnicodeStream, Int Bytes, BinaryStream, Long, Date, Object, Float, Time, BigDecimal,TimeStamp)
* A Noter : données volumineuses (ex. Blob)
  * Ouverture d'un flux

In [5]:
import java.util.List;
import java.util.ArrayList;
List<Film> films = new ArrayList<>();
while (resultSet.next()) {
        films.add(new Film(resultSet.getInt("film_id"),
                resultSet.getString("title"),
                resultSet.getString("description")));
}
System.out.println(films);

[{id:133,title:'Chamber Italian',description:'A Fateful Reflection of a Moose And a Husband who must Overcome a Monkey in Nigeria'}, {id:384,title:'Grosse Wonderful',description:'A Epic Drama of a Cat And a Explorer who must Redeem a Moose in Australia'}, {id:8,title:'Airport Pollock',description:'A Epic Tale of a Moose And a Girl who must Confront a Monkey in Ancient India'}, {id:98,title:'Bright Encounters',description:'A Fateful Yarn of a Lumberjack And a Feminist who must Conquer a Student in A Jet Boat'}, {id:1,title:'Academy Dinosaur',description:'A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies'}]


In [6]:
//Insert a city in France (34)
//Warning city names are not unique.
Statement statement = connection.createStatement();
String query1 = "INSERT INTO city(city, country_id)"+
    " VALUES('Garéoult',34)";
int numberOfChanges = statement.executeUpdate(query1);

## Les exceptions
* Erreur dans le code SQL : SQLException
* Avertissement lors de l'exécution (SQLWarning)
  * Problèmes de conversion de données (DataTruncation - sous-classe de SQLWarning)

In [7]:
//Traitement d'une erreur SQL
import java.sql.SQLException;
String wrongQuery = " SELECT * FROM Employee" ;
try (Connection connection = DriverManager.getConnection(jdbcURL);
            Statement statement = connection.createStatement();
            ResultSet resultSet = statement.executeQuery(wrongQuery)) {
        // Do stuff here
        } catch (SQLException e) {
            //Erreur lors de la requête
            e.printStackTrace();            
        }

org.postgresql.util.PSQLException: ERROR: relation "employee" does not exist
  Position: 16
	at org.postgresql.core.v3.QueryExecutorImpl.receiveErrorResponse(QueryExecutorImpl.java:2578)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2313)
	at org.postgresql.core.v3.QueryExecutorImpl.execute(QueryExecutorImpl.java:331)
	at org.postgresql.jdbc.PgStatement.executeInternal(PgStatement.java:448)
	at org.postgresql.jdbc.PgStatement.execute(PgStatement.java:369)
	at org.postgresql.jdbc.PgStatement.executeWithFlags(PgStatement.java:310)
	at org.postgresql.jdbc.PgStatement.executeCachedSql(PgStatement.java:296)
	at org.postgresql.jdbc.PgStatement.executeWithFlags(PgStatement.java:273)
	at org.postgresql.jdbc.PgStatement.executeQuery(PgStatement.java:226)
	at REPL.$JShell$39.do_it$($JShell$39.java:22)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMeth

## Un exemple complet

Voilà un exemple complet avec la classique des exceptions. Attention, quand une connection n'est plus utilisée il faut la fermer pour libérer les ressources sur le SGBD.

In [8]:
 String jdbcURL = "jdbc:postgresql://db:5432/dvdrental?user=postgres&password=changeme";

        Connection connection = null;
        ResultSet resultSet = null;
        Statement statement = null;
        try {
            connection = DriverManager.getConnection(jdbcURL);
            statement = connection.createStatement();
            String query1 = "SELECT * FROM film LIMIT 5";
            resultSet = statement.executeQuery(query1);

            List<Film> films = new ArrayList<>();
            while (resultSet.next()) {
                films.add(new Film(resultSet.getInt("film_id"),
                        resultSet.getString("title"),
                        resultSet.getString("description")));
            }
            System.out.println(films);
        } catch (SQLException e) {
            //Erreur lors de la requête
            e.printStackTrace();
        } finally {
            if (resultSet != null) {
                try {
                    resultSet.close();
                } catch (SQLException e) { /* ignored */}
            }
            if (connection != null) {
                try {
                    connection.close();
                } catch (SQLException e) { /* ignored */}
            }
        }

[{id:133,title:'Chamber Italian',description:'A Fateful Reflection of a Moose And a Husband who must Overcome a Monkey in Nigeria'}, {id:384,title:'Grosse Wonderful',description:'A Epic Drama of a Cat And a Explorer who must Redeem a Moose in Australia'}, {id:8,title:'Airport Pollock',description:'A Epic Tale of a Moose And a Girl who must Confront a Monkey in Ancient India'}, {id:98,title:'Bright Encounters',description:'A Fateful Yarn of a Lumberjack And a Feminist who must Conquer a Student in A Jet Boat'}, {id:1,title:'Academy Dinosaur',description:'A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies'}]


A partir de Java 7 une syntaxe particulière (try-with-ressources) est possible pour fermer automatiquement certaines ressources dont les connections et les ResultSet JDBC.

In [9]:
  String jdbcURL = "jdbc:postgresql://db:5432/dvdrental?user=postgres&password=changeme";

        String query1 = "SELECT * FROM film LIMIT 5";
        List<Film> films = new ArrayList<>();

        try (Connection connection = DriverManager.getConnection(jdbcURL);
            Statement statement = connection.createStatement();
            ResultSet resultSet = statement.executeQuery(query1)) {
            while (resultSet.next()) {
                films.add(new Film(resultSet.getInt("film_id"),
                        resultSet.getString("title"),
                        resultSet.getString("description")));
            }
            System.out.println(films);
        } catch (SQLException e) {
            //Erreur lors de la requête
            e.printStackTrace();
        }

[{id:133,title:'Chamber Italian',description:'A Fateful Reflection of a Moose And a Husband who must Overcome a Monkey in Nigeria'}, {id:384,title:'Grosse Wonderful',description:'A Epic Drama of a Cat And a Explorer who must Redeem a Moose in Australia'}, {id:8,title:'Airport Pollock',description:'A Epic Tale of a Moose And a Girl who must Confront a Monkey in Ancient India'}, {id:98,title:'Bright Encounters',description:'A Fateful Yarn of a Lumberjack And a Feminist who must Conquer a Student in A Jet Boat'}, {id:1,title:'Academy Dinosaur',description:'A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies'}]


## Types  Java/JDBC et SQL
* Malgré SQL les SGBD présentent des différences de types
* JDBC masque ces différences en définissant ses propres types SQL (constantes de la classe Types)
* Le driver assure la conversion
  * SQL vers Java lors de la lecture
  * Java vers SQL lors du passage de paramètres
* Utilisés explicitement avec les methodes getXXX() (et setXXX())
* Parfois plusieurs choix (presque tous les types SQL peuvent être retrouvés par getString())
  * CHAR et VARCHAR : getString()
  * LONGVARCHAR : getAsciiStream() et getCharacterStream()
  * BINARY et VARBINARY : getBytes()
  * LONGVARBINARY : getBinaryStream()
  * REAL : getFloat(), DOUBLE et FLOAT : getDouble()
  * DECIMAL et NUMERIC : getBigDecimal()
  * DATE : getDate(), TIME : getTime(), TIMESTAMP :getTimestamp()

* Par défaut une connexion est ouverte en « auto-
commit » :
  * un commit est automatiquement lancé après chaque requete SQL qui fait une mise à jour
* Pour un contrôle plus fin on utilise
```java
conn.setAutoCommit(false) pour le désactiver
conn.commit() pour valider la transaction
conn.rollback() pour annuler la transaction
```

## Précompilation des requêtes
* Si les requêtes fabriquées à partir de String changent (paramètres) :
  * Elles sont compilées à chaque appel d'où une perte de
performances
* JDBC permet de ne compiler la requête qu'une fois (si le
SGBD le supporte)
  * En indiquant les paramètres de façon générique
  * En fixant leur valeur (sans changer la requête) au moment
de l'exécution
* Deux Statement particuliers :
  * Les requêtes paramétrées (PreparedStatement)
Les procédures stockées (CallableStatement)

In [10]:
import java.sql.PreparedStatement;
Connection connection = DriverManager.getConnection(jdbcURL);
PreparedStatement preparedStatement = connection.prepareStatement("SELECT * FROM film WHERE film_id = ?");
int[] ids={23,12,56};
List<Film> films = new ArrayList<>();
//Juste pour illuster le cas ou une requête revient fréquement, mieux vaudrait une seule requêtes SQL
for(int id:ids) {
    preparedStatement.setInt(1,id);
    ResultSet resultSet = preparedStatement.executeQuery();
            while (resultSet.next()) {
                films.add(new Film(resultSet.getInt("film_id"),
                        resultSet.getString("title"),
                        resultSet.getString("description")));
            }
}
System.out.println(films);
    

[{id:23,title:'Anaconda Confessions',description:'A Lacklusture Display of a Dentist And a Dentist who must Fight a Girl in Australia'}, {id:12,title:'Alaska Phantom',description:'A Fanciful Saga of a Hunter And a Pastry Chef who must Vanquish a Boy in Australia'}, {id:56,title:'Barefoot Manchurian',description:'A Intrepid Story of a Cat And a Student who must Vanquish a Girl in An Abandoned Amusement Park'}]


## Les transactions

In [11]:
Connection connection = DriverManager.getConnection(jdbcURL);


//Create a account table
String createAccountTableSql="CREATE TABLE account ("+
    "id SERIAL PRIMARY KEY,"+
    "name VARCHAR(100) NOT NULL,"+
    "balance DEC(15,2) NOT NULL)";
Statement statement = connection.createStatement();
statement.executeUpdate(createAccountTableSql);
statement.close();

    
//Create accounts for Alice and Bob with a prepared statement
String createAccountSql = "INSERT INTO account(name,balance) VALUES(?,?);";
PreparedStatement pstmt = connection.prepareStatement(createAccountSql);
pstmt.setString(1,"Bob");
pstmt.setInt(2, 1000);
pstmt.executeUpdate();

pstmt.setString(1,"Alice");
pstmt.setInt(2, 1000);
pstmt.executeUpdate();

pstmt.close();

PreparedStatement pstmtIncreaseAccount = connection.prepareStatement("UPDATE account SET balance = balance + ? WHERE id = ?");

//save autoCommit state
boolean autoCommit = connection.getAutoCommit();
try {
    connection.setAutoCommit(false);
    //remove 500€ to Bob
    pstmtIncreaseAccount.setInt(1,-500);
    pstmtIncreaseAccount.setInt(2,1);
    pstmtIncreaseAccount.executeUpdate();
    
    //add 500€ to Alice
    pstmtIncreaseAccount.setInt(1,500);
    pstmtIncreaseAccount.setInt(2,2);
    pstmtIncreaseAccount.executeUpdate();
    connection.commit();
} catch (SQLException exc) {
    //Cancel the whole transaction if there is a problem.
    connection.rollback();
} finally {
    //restore autoCommit state
    connection.setAutoCommit(autoCommit);
}

//Drop the table
connection.createStatement().executeUpdate("DROP TABLE account");

connection.close();

## Procédure stockées
CallableStatement permet d'appeller une procédure stockée directement sur le SGBD. 
TODO

## Metadata
* JDBC permet de récupérer des informations sur le type de données que l'on vient de récupérer par un SELECT (interface ResultSetMetaData),
* mais aussi sur la base de données elle-même (interface DatabaseMetaData)
* Les données que l'on peut récupérer avec DatabaseMetaData dépendent du SGBD avec lequel on travaille

In [12]:
//Lecture des metadonnées du serveur de base de données
import java.sql.*;

String jdbcURL="jdbc:postgresql://db:5432/postgres?user=postgres&password=changeme";

try (Connection connection = DriverManager.getConnection(jdbcURL)) {

                DatabaseMetaData metadata = connection.getMetaData();

                //Print info about the database system
                System.out.println("Database: " + metadata.getDatabaseProductName()
                        + " " + metadata.getDatabaseMajorVersion() + "." + metadata.getDatabaseMinorVersion());

                System.out.println(connection.getCatalog() + " " + connection.getSchema());

                //Retrieving the list of database names
                ResultSet tables = metadata.getTables(connection.getCatalog(),
                        connection.getSchema(),
                        "SIMPLEJDBC_PERSON", null);

                if (tables.next()) {
                    System.out.println("Table " + tables.getString("TABLE_NAME") + " already exist.");
                } else {
                    connection.createStatement().execute("CREATE TABLE \"SIMPLEJDBC_PERSON\"(" +
                            "    id INT PRIMARY KEY NOT NULL, " +
                            "    firstname VARCHAR(100))");
                    System.out.println("Table SIMPLEJDBC_PERSON created.");
                }
            } catch (SQLException e) {
                e.printStackTrace();
            }


Database: PostgreSQL 9.6
postgres public
Table SIMPLEJDBC_PERSON created.


In [13]:
//Traitement des metadonnées d'un ResulSet
//pour la découverte du nombre, du type et du nom des colonnes. 
String jdbcURL="jdbc:postgresql://db:5432/dvdrental?user=postgres&password=changeme";

try (Connection connection = DriverManager.getConnection(jdbcURL);     
     ResultSet rs = connection.createStatement().executeQuery("SELECT * FROM film")) {

ResultSetMetaData rsmd = rs.getMetaData();
int nbColonnes = rsmd.getColumnCount();
for (int i = 1; i <= nbColonnes; i++) {
  String typeColonne = rsmd.getColumnTypeName(i);
  String nomColonne = rsmd.getColumnName(i);
  System.out.println("Colonne " + i + " de nom " + nomColonne + " de type " + typeColonne);
}
}

Colonne 1 de nom film_id de type serial
Colonne 2 de nom title de type varchar
Colonne 3 de nom description de type text
Colonne 4 de nom release_year de type int4
Colonne 5 de nom language_id de type int2
Colonne 6 de nom rental_duration de type int2
Colonne 7 de nom rental_rate de type numeric
Colonne 8 de nom length de type int2
Colonne 9 de nom replacement_cost de type numeric
Colonne 10 de nom rating de type mpaa_rating
Colonne 11 de nom last_update de type timestamp without time zone
Colonne 12 de nom special_features de type _text
Colonne 13 de nom fulltext de type tsvector


## ResultSet modifiables

In [14]:
import java.sql.Statement;
import java.sql.ResultSet;
import java.sql.DriverManager;
import java.sql.Connection;
String jdbcURL="jdbc:postgresql://db:5432/dvdrental?user=postgres&password=changeme";
Connection connection = DriverManager.getConnection(jdbcURL);
Statement statement = connection.createStatement(
  ResultSet.TYPE_SCROLL_INSENSITIVE, 
  ResultSet.CONCUR_UPDATABLE
);

//to be complete

## Datasources
Il est important de gérer l'ensemble des connexions qui sont ouvertes et idéalement de les réutiliser plutôt que de fermer/ouvrir. Cela peut être fait "à la main" en utilisant le concept de DataSource ou utilisant une librairie comme [Apache DBCP](http://commons.apache.org/proper/commons-dbcp/).

## Batch Update
Pour des ajouts en masse pensez à utiliser les batch updates mais attention 

In [15]:
// Création d'un requete
PreparedStatemen pstmt = 
  conn.prepareStatement("INSERT INTO EMPLOYE (email, nom, prenom) VALUES(?, ?, ?)");
 
//On ajoute les exécution une à une ...
conn.setAutoCommit(false);
 
// On fixe les paramètres de la première requête à exécuter
pstmt.setString( 1, "...@...");
pstmt.setString( 2, "..." );
pstmt.setString( 3, "..." );
// Et on l'ajoute au batch
pstmt.addBatch();
 
// On fixe les paramètres de la seconde requête à exécuter
pstmt.setString( 1, "...@...");
pstmt.setString( 2, "..." );
pstmt.setString( 3, "..." );
// Et on l'ajoute au batch
pstmt.addBatch();
 
// On ajoute autant de requêtes que nécessaire
//Par exemple en lisant un flux de données.
.
.
.
.
//On créée un tableau d'entiers pour recevoir les résultats.
//et on execute toutes les mises à jour en une fois.
int[] count = stmt.executeBatch();
 
//On valide les changements.
conn.commit();

CompilationException: 